In [1]:
import scanpy as sc
import anndata as ad
from scprocessing.Pipeline import Pipeline
from scprocessing.QC import QC
from scprocessing.Normalization import Normalization
from scprocessing.Integration import Integration
from scprocessing.metrics import jaccard, silhouette, davies, calinski, evaluate
from scprocessing.SelectPipeline import SelectPipeline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import rpy2.robjects as ro
import anndata2ri
anndata2ri.activate()

/tmp/ipykernel_18612/225481395.py:3: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [4]:
# paths
nsg_bus_1 = "/mnt/shared/nationwide/Counts/NSG_BUS_1/outs/filtered_feature_bc_matrix.h5"
nsg_bus_2 = "/mnt/shared/nationwide/Counts/NSG_BUS_2/outs/filtered_feature_bc_matrix.h5"
nsg_bus_3 = "/mnt/shared/nationwide/Counts/NSG_BUS_3/outs/filtered_feature_bc_matrix.h5"

nsg_ctl_1 = "/mnt/shared/nationwide/Counts/NSG_CNTL_1/outs/filtered_feature_bc_matrix.h5"
nsg_ctl_2 = "/mnt/shared/nationwide/Counts/NSG_CNTL_2/outs/filtered_feature_bc_matrix.h5"
nsg_ctl_3 = "/mnt/shared/nationwide/Counts/NSG_CNTL_3/outs/filtered_feature_bc_matrix.h5"

nsg_s_bus_1 = "/mnt/shared/nationwide/Counts/NSG_S_BUS_1/outs/filtered_feature_bc_matrix.h5"
nsg_s_bus_2 = "/mnt/shared/nationwide/Counts/NSG_S_BUS_2/outs/filtered_feature_bc_matrix.h5"
nsg_s_bus_3 = "/mnt/shared/nationwide/Counts/NSG_S_BUS_3/outs/filtered_feature_bc_matrix.h5"

nsg_s_ctl_1 = "/mnt/shared/nationwide/Counts/NSG_S_CNTL_1/outs/filtered_feature_bc_matrix.h5"
nsg_s_ctl_2 = "/mnt/shared/nationwide/Counts/NSG_S_CNTL_2/outs/filtered_feature_bc_matrix.h5"
nsg_s_ctl_3 = "/mnt/shared/nationwide/Counts/NSG_S_CNTL_3/outs/filtered_feature_bc_matrix.h5"

In [5]:
# read data
nsg_bus_1_data = sc.read_10x_h5(nsg_bus_1)
nsg_bus_2_data = sc.read_10x_h5(nsg_bus_2)
nsg_bus_3_data = sc.read_10x_h5(nsg_bus_3)

# creating metadata
nsg_bus_1_data.obs["Trial"] = "1"
nsg_bus_2_data.obs["Trial"] = "2"
nsg_bus_3_data.obs["Trial"] = "3"

# making names unique
nsg_bus_1_data.var_names_make_unique()
nsg_bus_2_data.var_names_make_unique()
nsg_bus_3_data.var_names_make_unique()

nsg_bus_1_data.obs_names_make_unique()
nsg_bus_2_data.obs_names_make_unique()
nsg_bus_3_data.obs_names_make_unique()

# nsg_bus_1_data = qc(nsg_bus_1_data)
# nsg_bus_2_data = qc(nsg_bus_2_data)
# nsg_bus_3_data = qc(nsg_bus_3_data)

/home/user/.conda/envs/python3.8/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/user/.conda/envs/python3.8/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/user/.conda/envs/python3.8/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/user/.conda/envs/python3.8/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/user/.conda/envs/python3.8/lib/python3.10/site-packages/anndata/_core/anndata.py:1820: Use

In [6]:
human_brca_immune = sc.read_h5ad("/mnt/shared/nationwide/cell_type_datasets/human_brca_immune.h5ad")
human_brca_immune.var_names_make_unique()
human_brca_immune.obs_names_make_unique()
del human_brca_immune.obsm["X_diffmap"]
human_brca_immune.var.drop("feature_length", axis=1, inplace=True)

In [7]:
cond1 = "30-year-old human stage"
cond2 = "31-year-old human stage"
cond3 = "33-year-old human stage"
cond4 = "37-year-old human stage"
cond5 = "39-year-old human stage"
brca_group_1 = human_brca_immune[human_brca_immune.obs["development_stage"] == cond1]
brca_group_2 = human_brca_immune[human_brca_immune.obs["development_stage"] == cond2]
brca_group_3 = human_brca_immune[human_brca_immune.obs["development_stage"] == cond3]
brca_group_4 = human_brca_immune[human_brca_immune.obs["development_stage"] == cond4]
brca_group_5 = human_brca_immune[human_brca_immune.obs["development_stage"] == cond5]

brca_group_1.obs["Type"] = cond1
brca_group_2.obs["Type"] = cond2
brca_group_3.obs["Type"] = cond3
brca_group_4.obs["Type"] = cond4
brca_group_5.obs["Type"] = cond5

brca_group_1.obs["Type"] = brca_group_1.obs["Type"].astype("category")
brca_group_2.obs["Type"] = brca_group_2.obs["Type"].astype("category")
brca_group_3.obs["Type"] = brca_group_3.obs["Type"].astype("category")
brca_group_4.obs["Type"] = brca_group_4.obs["Type"].astype("category")
brca_group_5.obs["Type"] = brca_group_5.obs["Type"].astype("category")

/tmp/ipykernel_18612/4216058113.py:12: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  brca_group_1.obs["Type"] = cond1
/tmp/ipykernel_18612/4216058113.py:13: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  brca_group_2.obs["Type"] = cond2
/tmp/ipykernel_18612/4216058113.py:14: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  brca_group_3.obs["Type"] = cond3
/tmp/ipykernel_18612/4216058113.py:15: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  brca_group_4.obs["Type"] = cond4
/tmp/ipykernel_18612/4216058113.py:16: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  brca_group_5.obs["Type"] = cond5


In [8]:
def find_non_string_categorical_columns(adata):
    non_string_categorical_obs = []
    non_string_categorical_var = []

    # Check .obs
    for column in adata.obs.columns:
        if adata.obs[column].dtype.name == 'category' and not all(isinstance(x, str) for x in adata.obs[column].cat.categories):
            non_string_categorical_obs.append(column)

    # Check .var
    for column in adata.var.columns:
        if adata.var[column].dtype.name == 'category' and not all(isinstance(x, str) for x in adata.var[column].cat.categories):
            non_string_categorical_var.append(column)

    return non_string_categorical_obs, non_string_categorical_var

In [9]:
ro.globalenv["adata"] = human_brca_immune
ro.r("library(Seurat)")
ro.r("library(SeuratObject)")
ro.r("library(sctransform)")
ro.r("library(SeuratData)")
ro.r("library(glmGamPoi)")
ro.r("library(clustree)")
ro.r("library(ggplot2)")

/home/user/.conda/envs/python3.8/lib/python3.10/site-packages/rpy2/robjects/conversion.py:28: DeprecationWarning: The use of py2rpy in module rpy2.robjects.conversion is deprecated. Use rpy2.robjects.conversion.get_conversion() instead of rpy2.robjects.conversion.converter.
  warnings.warn(
R[write to console]: Loading required package: SeuratObject

R[write to console]: Loading required package: sp

R[write to console]: 
Attaching package: ‘sp’


R[write to console]: The following object is masked from ‘package:IRanges’:

    %over%


R[write to console]: 
Attaching package: ‘SeuratObject’


R[write to console]: The following object is masked from ‘package:SummarizedExperiment’:

    Assays


R[write to console]: The following object is masked from ‘package:GenomicRanges’:

    intersect


R[write to console]: The following object is masked from ‘package:GenomeInfoDb’:

    intersect


R[write to console]: The following object is masked from ‘package:IRanges’:

    intersect


R[write


    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: 
Attaching package: ‘Seurat’


R[write to console]: The following object is masked from ‘package:SummarizedExperiment’:

    Assays


R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: 2: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages

R[write to console]: 3: 
R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ co

In [10]:
ro.r("data <- as.Seurat(adata, counts='X', data=NULL)")

R[write to console]: Warning:
R[write to console]:  Keys should be one or more alphanumeric characters followed by an underscore, setting key from X_scVI_ to XscVI_



In [11]:
ro.r("data.list = SplitObject(data, split.by='development_stage')")

In [12]:
ro.r("target_data = list(data.list$`31-year-old human stage`, data.list$`30-year-old human stage`)")

In [13]:
ro.r("target_data <- lapply(target_data, function(x) SCTransform(x, assay='originalexp', method='glmGamPoi'))")

R[write to console]: Running SCTransform on assay: originalexp

R[write to console]: vst.flavor='v2' set. Using model with fixed slope and excluding poisson genes.

R[write to console]: Calculating cell attributes from input UMI matrix: log_umi

R[write to console]: Variance stabilizing transformation of count matrix of size 13249 by 1511

R[write to console]: Model formula is y ~ log_umi

R[write to console]: Get Negative Binomial regression parameters per gene

R[write to console]: Using 2000 genes, 1511 cells

R[write to console]: Found 3 outliers - those will be ignored in fitting/regularization step


R[write to console]: Second step: Get residuals using fitted parameters for 13249 genes

R[write to console]: Computing corrected count matrix for 13249 genes

R[write to console]: Calculating gene attributes

R[write to console]: Wall clock passed: Time difference of 7.785311 secs

R[write to console]: Determine variable features

R[write to console]: Centering data matrix

  |     

In [14]:
ro.r("features <- SelectIntegrationFeatures(object.list = target_data, nfeatures = 3000)")

In [15]:
ro.r("print(length(target_data[[1]][['SCT']]$scale.data))")

[1] 4533000


In [17]:
ro.r("print(length(features))")

[1] 3000


In [39]:
ro.r("target_data <- PrepSCTIntegration(object.list = target_data, assay='SCT', anchor.features = features)")

  |                                                  | 0 % ~calculating  

R[write to console]: Error in scale.data[anchor.features, ] : subscript out of bounds



RRuntimeError: Error in scale.data[anchor.features, ] : subscript out of bounds


In [155]:
ro.r("adata <- lapply(adata, function(x) ScaleData(x))")

R[write to console]: Centering and scaling data matrix

  |                                                                            
  |                                                                      |   0%
  |                                                                            
  |===================================                                   |  50%
  |                                                                            
  |======================================================================| 100%
R[write to console]: 

R[write to console]: Centering and scaling data matrix

  |                                                                            
  |                                                                      |   0%
  |                                                                            
  |===================================                                   |  50%
  |                                                              

In [156]:
ro.r("adata <- lapply(adata, function(x) RunPCA(x, npcs = 30))")

R[write to console]: PC_ 1 
Positive:  ENSG00000152518, ENSG00000133872, ENSG00000110848, ENSG00000173762, ENSG00000157514, ENSG00000168685, ENSG00000168028, ENSG00000081237, ENSG00000121966, ENSG00000151883 
	   ENSG00000101596, ENSG00000175061, ENSG00000198755, ENSG00000211772, ENSG00000196405, ENSG00000265972, ENSG00000145675, ENSG00000111796, ENSG00000134539, ENSG00000153563 
	   ENSG00000235576, ENSG00000115875, ENSG00000181163, ENSG00000198851, ENSG00000116824, ENSG00000160593, ENSG00000274020, ENSG00000107742, ENSG00000245910, ENSG00000134242 
Negative:  ENSG00000158869, ENSG00000204287, ENSG00000198502, ENSG00000176788, ENSG00000197405, ENSG00000159189, ENSG00000166927, ENSG00000216490, ENSG00000173369, ENSG00000079215 
	   ENSG00000151726, ENSG00000196735, ENSG00000115919, ENSG00000130203, ENSG00000118257, ENSG00000125538, ENSG00000126353, ENSG00000196126, ENSG00000277443, ENSG00000011600 
	   ENSG00000157557, ENSG00000143162, ENSG00000146070, ENSG00000135047, ENSG00000011422,

In [157]:
ro.r("adata <- lapply(adata, function(x) FindNeighbors(x, k.parm = 30))")

R[write to console]: Warning:
R[write to console]:  The following arguments are not used: k.parm

R[write to console]: Warning:
R[write to console]:  The following arguments are not used: k.parm

R[write to console]: Computing nearest neighbor graph

R[write to console]: Computing SNN

R[write to console]: Warning:
R[write to console]:  The following arguments are not used: k.parm

R[write to console]: Warning:
R[write to console]:  The following arguments are not used: k.parm

R[write to console]: Computing nearest neighbor graph

R[write to console]: Computing SNN



In [158]:
ro.r("adata <- lapply(adata, function(x) FindClusters(x))")

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 977
Number of edges: 35625

Running Louvain algorithm...


R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: 

Maximum modularity in 10 random starts: 0.7260
Number of communities: 7
Elapsed time: 0 seconds
Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 1511
Number of edges: 52486

Running Louvain algorithm...


R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: 

Maximum modularity in 10 random starts: 0.8409
Number of communities: 8
Elapsed time: 0 seconds


In [159]:
ro.r("adata <- lapply(adata, function(x) RunUMAP(x, dims = 1:30))")

R[write to console]: 04:27:32 UMAP embedding parameters a = 0.9922 b = 1.112

R[write to console]: 04:27:32 Read 977 rows and found 30 numeric columns

R[write to console]: 04:27:32 Using Annoy for neighbor search, n_neighbors = 30

R[write to console]: 04:27:32 Building Annoy index with metric = cosine, n_trees = 50

R[write to console]: 0%   10   20   30   40   50   60   70   80   90   100%

R[write to console]: [----|----|----|----|----|----|----|----|----|----|

R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *
R[write to console]: *


In [160]:
ro.r("int_feats <- SelectIntegrationFeatures(adata)")

In [161]:
ro.r("print(adata)")

$data1
An object of class Seurat 
46463 features across 977 samples within 2 assays 
Active assay: SCT (12008 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: originalexp
 4 dimensional reductions calculated: X_scVI, UMAP, pca, umap

$data2
An object of class Seurat 
47704 features across 1511 samples within 2 assays 
Active assay: SCT (13249 features, 2000 variable features)
 3 layers present: counts, data, scale.data
 1 other assay present: originalexp
 4 dimensional reductions calculated: X_scVI, UMAP, pca, umap



In [162]:
ro.r("int_list <- PrepSCTIntegration(object.list = adata, anchor.features = int_feats, verbose=T)")

  |                                                  | 0 % ~calculating  

R[write to console]: Error in scale.data[anchor.features, ] : subscript out of bounds



RRuntimeError: Error in scale.data[anchor.features, ] : subscript out of bounds


In [ ]:
int_feats <- SelectIntegrationFeatures(seurat_objs)
int_list <- PrepSCTIntegration(object.list = seurat_objs,
                               anchor.features = int_feats)
int_anchors <- FindIntegrationAnchors(object.list = int_list,
                                      normalization.method = "SCT",
                                      anchor.features = int_feats)
cca <- IntegrateData(anchorset = int_anchors,
                     normalization.method = "SCT")
remove(int_anchors, int_list, int_feats)
cca <- cca %>%
  RunPCA(verbose = FALSE) %>%
  FindNeighbors(dims = 1:30) %>%
  RunUMAP(dims = 1:30) %>%
  FindClusters()
resolution.range <- seq(from = 0, to = 1, by = 0.1)
cca <- FindClusters(cca, resolution = resolution.range)
tree <- clustree(cca)
cca <- FindClusters(cca, resolution = 0.3)